In [0]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

In [0]:
#train ve test verilerini train ve test olarak okuduk
train=pd.read_excel("clean_tweet_train.xlsx")
test=pd.read_excel("clean_tweet_test.xlsx")

In [0]:
corpus = train.append(test, ignore_index=True).fillna(' ')

In [0]:
test.dropna(inplace=True)
test.reset_index(drop=True,inplace=True)

train.dropna(inplace=True)
train.reset_index(drop=True,inplace=True)

In [0]:
x_train=train.text
y_train=train.sentiment
x_test=test.text
y_test=test.sentiment

In [0]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [0]:

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

Using TensorFlow backend.


In [0]:
#Her tweet'i unique bir ID ile etiketliyoruz
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [0]:
#bütün tweet verilerini topladık -train-test olan text columnlar  toplandı
all_x = pd.concat([x_train,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [0]:
#tweet kelimelerine word2vec cbow yöntemi(sg=0) uygulanıyor,
#cümle içindeki current_wod ile predicted word arasındaki mesafe window_size=2
#size=100 feature vetörlerin boyutu
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 17287/17287 [00:00<00:00, 584417.56it/s]


In [0]:
#embedding eğitimi yapılıyor
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 17287/17287 [00:00<00:00, 899757.19it/s]


In [0]:
#daha sonra skip-gram modeli 
model_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 17287/17287 [00:00<00:00, 636835.74it/s]


In [0]:
%%time
#kelime vektörlerinin elde edilemesi için skip-gram modeli kullanılıyor
for epoch in range(30):
    model_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_sg.alpha -= 0.002
    model_sg.min_alpha = model_sg.alpha

100%|██████████| 17287/17287 [00:00<00:00, 988034.79it/s]


CPU times: user 22.9 s, sys: 233 ms, total: 23.1 s
Wall time: 25.1 s


In [0]:
#bu iki yöntemi birleştirdik
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_sg.wv[w])
print('Word vektör sayısı:' , len(embeddings_index))

Word vektör sayısı: 12029


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#tokenizer ile  cümleydeki kelimeleri bölüyoruz
#her cümlenin sequential gösterimi için text_to_sequences kullanıyoruz
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

print("Toplam kelime sayısı-train verisindeki:",len(tokenizer.word_index))

Toplam kelime sayısı-train verisindeki: 26543


In [0]:
length = []
for x in corpus['text']:#bütün veri setindeki max kelime saysını bulmak için
    length.append(len(x.split()))
#padding için en uzun cümledeki kelime sayısını bulduk
print("max kelime sayısı,cümledeki")
max(length)

max kelime sayısı,cümledeki


33

In [0]:
#train veri setindeki bütün cümleler 35 uzunluğuna çevrildi ,0 padding yapıldı
x_train_seq = pad_sequences(sequences, maxlen=35)
x_train_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    4,   78, 3207,    1,
        3208, 1603],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  919,    1,   25,   26,   12,  252,    2,
          28, 4856],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,
           7,  300],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    1, 4857,
         293,   41],
       [   0,    0,    0,   

In [0]:
#test veri setindeki bütün cümleler 35 uzunluğuna çevrildi ,0 padding yapıldı
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=35)

In [0]:
#elde ettiğimix-z kelime vektörlerinden bir matrix oluşturuyoruz ,embedding layer
#için num_words ile training için kullanacağımız most frequent word sayısı belirlendi
#200 ise embedding_dimension 
num_words = 10000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
#üç sınıflı bir veri setimiz olduğu için  3 
from keras import utils as np_utils
y_test = np_utils.to_categorical(y_test, num_classes= 3)
y_train = np_utils.to_categorical(y_train, num_classes= 3)
print("y_train görünümü:")
print(y_train)

y_train görünümü:
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [0]:
print(x_train_seq.shape)
print(y_train.shape)
print(x_test_seq.shape)
print(y_test.shape)

(13831, 35)
(13831, 3)
(3456, 35)
(3456, 3)


In [0]:
max_features = 10000#training için most frequent 10.000 kelime kullanılacak
maxlen =35#en uzun cümledeki kelime sayısı+2
embedding_dims = 200#output vektör size,her cümle 35*200 matrix ile ifade edilecek,ayrıca filter column genişliği
filters = 32#dimensionality of the output 
kernel_size = 3#window size uzunluğu
hidden_dims = 64

In [0]:
#ilk durumda word embeddingler embedding layerdan elde edildi
model = Sequential()
#embedding layer ile vocab indexleri embedding dimensions'lara çeviriyor
model.add(Embedding(max_features, embedding_dims,input_length=maxlen))
#dropout overfitting'i önlemek için kullanıldı
model.add(Dropout(0.2))

#konvolüsyon katmanı,stride 1 strides vertically
model.add(Conv1D(filters,kernel_size,padding='same',activation='relu',strides=1))

# max pooling katmanında ;output dimensiondaki her filtreden max olanı alır,1 boyutlu
#bir vektör elde etmek için ,uzunluğu filtre sayısı ile aynıdır
model.add(GlobalMaxPooling1D())

#  hidden layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# output layer aktivasyon 'softmax'
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 200)           2000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 35, 32)            19232     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
__________

In [0]:
model.fit(x_train_seq, y_train, epochs=10, batch_size=32)
scores = model.evaluate(x_test_seq, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
13831/13831 [==============================] - 9s 624us/step - loss: 0.8984 - acc: 0.5765
Epoch 2/10
13831/13831 [==============================] - 6s 435us/step - loss: 0.5326 - acc: 0.7924
Epoch 3/10
13831/13831 [==============================] - 6s 443us/step - loss: 0.2703 - acc: 0.9065
Epoch 4/10
13831/13831 [==============================] - 6s 434us/step - loss: 0.1510 - acc: 0.9530
Epoch 5/10
13831/13831 [==============================] - 6s 428us/step - loss: 0.1040 - acc: 0.9706
Epoch 6/10
13831/13831 [==============================] - 6s 431us/step - loss: 0.0780 - acc: 0.9766
Epoch 7/10
13831/13831 [==============================] - 6s 424us/step - loss: 0.0677 - acc: 0.9800
Epoch 8/10
13831/13831 [==============================] - 6s 433us/step - loss: 0.0562 - acc: 0.9825
Epoch 9/10
13831/13831 [==============================] - 6s 421us/step - loss: 0.0449 - acc: 0.9845
Epoch 10/10
13831/13831 [==============================] - 5s 364us/step - loss: 0.0412 - a

In [0]:
#ikinci durumda kelime vektörleri Word2vec ten elde edildi
model = Sequential()
#embedding layer ile vocab indexleri embedding dimensions'lara çeviriyor
model.add(Embedding(max_features, embedding_dims, weights=[embedding_matrix],input_length=maxlen,trainable=False))
#dropout overfitting'i önlemek için kullanıldı
model.add(Dropout(0.2))

#konvolüsyon katmanı,stride 1 strides vertically
model.add(Conv1D(filters,kernel_size,padding='same',activation='relu',strides=1))

# max pooling katmanında ;output dimensiondaki her filtreden max olanı alır,1 boyutlu
#bir vektör elde etmek için ,uzunluğu filtre sayısı ile aynıdır
model.add(GlobalMaxPooling1D())

#  hidden layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# output layer aktivasyon 'softmax'
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 200)           2000000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 35, 200)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 32)            19232     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
__________

In [0]:
model.fit(x_train_seq, y_train, epochs=10, batch_size=32)
scores = model.evaluate(x_test_seq, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
13831/13831 [==============================] - 6s 410us/step - loss: 0.9761 - acc: 0.5413
Epoch 2/10
13831/13831 [==============================] - 5s 362us/step - loss: 0.8310 - acc: 0.6287
Epoch 3/10
13831/13831 [==============================] - 5s 364us/step - loss: 0.7910 - acc: 0.6483
Epoch 4/10
13831/13831 [==============================] - 5s 349us/step - loss: 0.7591 - acc: 0.6646
Epoch 5/10
13831/13831 [==============================] - 5s 355us/step - loss: 0.7346 - acc: 0.6740
Epoch 6/10
13831/13831 [==============================] - 5s 361us/step - loss: 0.7144 - acc: 0.6853
Epoch 7/10
13831/13831 [==============================] - 5s 351us/step - loss: 0.6884 - acc: 0.6971
Epoch 8/10
13831/13831 [==============================] - 5s 364us/step - loss: 0.6712 - acc: 0.7095
Epoch 9/10
13831/13831 [==============================] - 5s 354us/step - loss: 0.6563 - acc: 0.7146
Epoch 10/10
13831/13831 [==============================] - 5s 364us/step - loss: 0.6447 - a

In [0]:
#üçüncü durumda word2vec kelime vektörleri training sırasında update edildi
model = Sequential()
#embedding layer ile vocab indexleri embedding dimensions'lara çeviriyor
model.add(Embedding(max_features, embedding_dims, weights=[embedding_matrix],input_length=maxlen,trainable=True))
#dropout overfitting'i önlemek için kullanıldı
model.add(Dropout(0.2))

#konvolüsyon katmanı,stride 1 strides vertically
model.add(Conv1D(filters,kernel_size,padding='same',activation='relu',strides=1))

# max pooling katmanında ;output dimensiondaki her filtreden max olanı alır,1 boyutlu
#bir vektör elde etmek için ,uzunluğu filtre sayısı ile aynıdır
model.add(GlobalMaxPooling1D())

#  hidden layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# output layer aktivasyon 'softmax'
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 35, 200)           2000000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 35, 200)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 32)            19232     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_5 (Activation)    (None, 64)                0         
__________

In [0]:
model.fit(x_train_seq, y_train, epochs=10, batch_size=32)
scores = model.evaluate(x_test_seq, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
13831/13831 [==============================] - 7s 473us/step - loss: 0.9297 - acc: 0.5653
Epoch 2/10
13831/13831 [==============================] - 3s 246us/step - loss: 0.7812 - acc: 0.6532
Epoch 3/10
13831/13831 [==============================] - 3s 246us/step - loss: 0.6905 - acc: 0.7010
Epoch 4/10
13831/13831 [==============================] - 3s 244us/step - loss: 0.6055 - acc: 0.7478
Epoch 5/10
13831/13831 [==============================] - 3s 246us/step - loss: 0.5312 - acc: 0.7806
Epoch 6/10
13831/13831 [==============================] - 3s 244us/step - loss: 0.4520 - acc: 0.8193
Epoch 7/10
13831/13831 [==============================] - 3s 245us/step - loss: 0.3847 - acc: 0.8502
Epoch 8/10
13831/13831 [==============================] - 3s 244us/step - loss: 0.3308 - acc: 0.8712
Epoch 9/10
13831/13831 [==============================] - 3s 244us/step - loss: 0.2814 - acc: 0.8905
Epoch 10/10
13831/13831 [==============================] - 3s 246us/step - loss: 0.2371 - a